In [1]:
import urdf2casadi.urdfparser as u2c
import numpy as np

## 1. Load robot from urdf 

1. Create urdfparser-class instance. 
2. Load model to instance, either from file, string or ros parameter server. Examples uses from file. 

In [4]:
ur5 = u2c.URDFparser()
ur5.from_file("/home/lillmaria/urdf2casadi/examples/urdf/ur5_mod.urdf")

/home/lillmaria/urdf2casadi/examples/urdf/ur5_mod.urdf


## 2. Get joint information ? 
Information about the joints of the robot model can be obtained using u2c's get_joint_info(). "root" and "tip" are the inputs, and represent the link names for the root and tip of the kinematic tree one wishes to evaluate. 

In [5]:
root = "base_link"
tip = "wrist_3_link"

joint_list, joint_names, q_max, q_min = ur5.get_joint_info(root, tip)
n_joints = len(joint_list)
print "name of first joint:", joint_names[0], "\n"
print "joint information for first joint:\n", joint_list[0]
print "\n q max:", q_max
print "\n q min:", q_min

name of first joint: shoulder_pan_joint 

joint information for first joint:
axis:
- 0.0
- 0.0
- 1.0
calibration: None
child: shoulder_link
dynamics:
  damping: 0.0
  friction: 0.0
limit:
  effort: 150.0
  lower: -6.28318530718
  upper: 6.28318530718
  velocity: 3.15
mimic: None
name: shoulder_pan_joint
origin:
  rpy:
  - 0.0
  - 0.0
  - 0.0
  xyz:
  - 0.0
  - 0.0
  - 0.089159
parent: base_link
safety_controller: None
type: revolute

 q max: [6.28318530718, 6.28318530718, 3.14159265359, 6.28318530718, 6.28318530718, 6.28318530718]

 q min: [-6.28318530718, -6.28318530718, -3.14159265359, -6.28318530718, -6.28318530718, -6.28318530718]


# 3. Obtain robot dynamics! 
After loading the robot model to the urdfparser-instance, the robot dynamics can be obtained from any given root and tip of the robot. 

## Dynamic parameters:
To obtain the dynamic parameters (M, C, G) of the equation of motion tau = M(q)q_ddot + C(q, q_dot)q_dot + G(q):

In [6]:
M_sym = ur5.get_inertia_matrix_crba(root, tip)
C_sym = ur5.get_coriolis_rnea(root, tip)

gravity = [0, 0, -9.81]
G_sym = ur5.get_gravity_rnea(root, tip, gravity)

M_sym, C_sym, G_sym are CasADi symbolic expressions of the ur5's dynamic parameters from given root to tip. The CasADi expressions are C-code generated so they can be numerically evaluated efficiently. Example of numerical evaluation is shown below. 

In [7]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    
M_num = M_sym(q)
C_num = C_sym(q, q_dot)
G_num = G_sym(q)
print "Numerical Inertia Matrx for random input: \n", M_num 
print "\nNumerical Coriolis term for random input: \n", C_num 
print "\nNumerical gravity term for random input: \n", G_num

Numerical Inertia Matrx for random input: 

[[1.69853, -0.0439708, -0.047046, 0.000340887, 0.238063, -0.00145248], 
 [-0.0439708, 1.53628, 0.307338, 0.245481, 0.00277671, -0.0160655], 
 [-0.047046, 0.307338, 0.848527, 0.247378, 0.00277671, -0.0160655], 
 [0.000340887, 0.245481, 0.247378, 0.241881, 0.00277671, -0.0160655], 
 [0.238063, 0.00277671, 0.00277671, 0.00277671, 0.247334, 0], 
 [-0.00145248, -0.0160655, -0.0160655, -0.0160655, 0, 0.0171365]]

Numerical Coriolis term for random input: 
[-0.185397, 0.379857, 0.469269, -0.00705312, -0.115715, -0.0340975]

Numerical gravity term for random input: 
[-1.11022e-16, 31.0673, -12.4185, -0.0424965, 0, 0]


## Inverse Dynamics 

Without accounting for gravitational forces:

In [8]:
tau_sym = ur5.get_inverse_dynamics_rnea(root, tip)

Accounting for gravitational forces:

In [9]:
gravity = [0, 0, -9.81]
tau_g_sym = ur5.get_inverse_dynamics_rnea(root, tip, gravity = gravity)

External forces can also be accounted for:

In [11]:
q = [None]*n_joints
q_dot = [None]*n_joints
q_ddot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_ddot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

tau_num = tau_sym(q, q_dot, q_ddot)
tau_g_num = tau_g_sym(q, q_dot, q_ddot)
#tau_fext_num = tau_fext_sym(q, q_dot, q_ddot)
print "Numerical inverse dynamics: \n", tau_num 
print "\nNumerical inverse dynamics w/ gravity: \n", tau_g_num 
#print "\nNumerical inverse dynamics w/ external forces: \n", G_num

Numerical inverse dynamics: 
[-15.1485, 6.45096, 14.7194, -2.66663, 6.61029, 0.422313]

Numerical inverse dynamics w/ gravity: 
[-15.1485, -27.2411, 24.4046, -2.50092, 6.61029, 0.422313]


## Forward Dynamics

urdf2casadi provides two methods for finding the robot's forward dynamics. The first method combines the recursive Newton-Euler algorithm (RNEA) and the composite rigid body algorithm (CRBA) and solves the equation of motion for the joint accelerations. The second method uses the articulated body algorithm (ABA) for forward dynamics. The method that uses ABA is in most cases the most efficient with regard to numerical evaluation, especially if the number of joints are high. (See timing examples for more information.)

In [18]:
tau = np.zeros(n_joints)
qddot_sym = ur5.get_forward_dynamics_crba(root, tip)

In [19]:
qddot_g_sym = ur5.get_forward_dynamics_aba(root, tip, gravity = gravity)


In [20]:
q = [None]*n_joints
q_dot = [None]*n_joints
for i in range(n_joints):
    #to make sure the inputs are within the robot's limits:
    q[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2
    q_dot[i] = (q_max[i] - q_min[i])*np.random.rand()-(q_max[i] - q_min[i])/2

qddot_num = qddot_sym(q, q_dot, tau)
qddot_g_num = qddot_g_sym(q, q_dot, tau)

print "Numerical inverse dynamics: \n", qddot_num 
print "\nNumerical inverse dynamics w/ gravity: \n", qddot_g_num 

Numerical inverse dynamics: 
[-25.6262, 18.6673, -38.13, 20.6464, 63.3116, -1.09492]

Numerical inverse dynamics w/ gravity: 
[-26.9289, 15.0076, -8.34084, -5.59211, 63.774, -0.444615]


# 4. Obtain the Derivatives 

From the dynamics functions, their derivatives can easily be obtained using CasADi`s built-in Jacobian functionality. The user can choose to find the derivative with regard to those variables needed (cs.jacobian()), or to find the time derivative with regard to these variables (cs.jtimes()), i.e the Jacobian times the time derivative of the variables.

If one are to find the time derivative, jtimes() is recommended over first obtaining the Jacobian, as jtimes() shortens the expressions, thus making the evaluation time of the expressions more efficient. 

Examples for obtaining the derivatives for the inverse dynamics, using both jacobian() and jtimes() are shown below:

## cs.jacobian()

The following explains how to use CasADi to obtain the derivative of the inverse dynamics with respect to q, qdot, and qddot, using the symbolic function returned by urdf2casadi (tau_sym):


1. Import CasADi and declare the symbolic variables needed in the derivative expression. 

In [33]:
import casadi as cs

q_sym =cs.SX.sym("qs", n_joints)
qdot_sym =cs.SX.sym("qsdot", n_joints)
qddot_sym =cs.SX.sym("qsddot", n_joints)

2. Declare the vector of the variables to find the derivatives with respect to using cs.vertcat():

In [39]:
id_vec = cs.vertcat(q_sym, qdot_sym, qddot_sym)
print id_vec

[qs_0, qs_1, qs_2, qs_3, qs_4, qs_5, qsdot_0, qsdot_1, qsdot_2, qsdot_3, qsdot_4, qsdot_5, qsddot_0, qsddot_1, qsddot_2, qsddot_3, qsddot_4, qsddot_5]


3. Obtain the symbolic expression of the derivative of ID with respect to q, qdot, and qddot using cs.jacobian():

In [41]:
derivative_id_expression = cs.jacobian(tau_sym(q_sym, qdot_sym, qddot_sym), id_vec)

4. Use the symbolic expression to make a CasADi function that can be efficiently numerical evaluated:

In [37]:
derivative_id_function  = cs.Function("did", [q_sym, qdot_sym, qddot_sym], [derivative_id_expression], {"jit": True, "jit_options":{"flags":"-Ofast"}})

where -Ofast flag is used to C-code generate the function. The derivative function can then be numerically evaluated similar to the functions returned by urdf2casadi, as illustrated in the above. For instance:

In [52]:
print derivative_id_function(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))


[[00, 3.22059, 0.652579, -1.94272, 0.107956, -0.174549, -1.40766, -0.953735, -0.27699, 0.0695818, 0.0794737, -0.00429257, 0.965012, 0.419042, 0.0753014, 0.00515581, 0.23959, -0.00203493], 
 [00, -1.80836, -3.38084, 0.531047, -0.0193066, -0.0290333, 1.23571, -1.11698, -2.25421, 0.0177435, -0.00648487, -0.0071203, 0.419042, 3.32406, 1.19369, 0.236788, 0.00638344, 0.00925888], 
 [00, -0.631776, -0.326539, 0.453458, -0.0193066, -0.0290333, 0.117541, 1.15117, 0.0139476, -0.00112916, -0.00648487, -0.0071203, 0.0753014, 1.19369, 0.833443, 0.243661, 0.00638344, 0.00925888], 
 [00, 0.452977, 0.442953, 0.405654, -0.0193066, -0.0290333, -0.100621, 0.0302718, 0.0365627, 0.0214859, -0.00648487, -0.0071203, 0.00515581, 0.236788, 0.243661, 0.249531, 0.00638344, 0.00925888], 
 [00, -0.720317, -0.720317, -0.720317, 0.118444, -0.143756, 0.0677002, 0.00299667, 0.00299667, 0.00299667, -0.0151721, -0.00368425, 0.23959, 0.00638344, 0.00638344, 0.00638344, 0.241427, 00], 
 [00, 0.0500702, 0.0500702, 0.05007


One can also find the derivative with respect to just one variable:


In [50]:
derivative_id_expression_dq = cs.jacobian(tau_sym(q_sym, qdot_sym, qddot_sym), q_sym)
derivative_id_function_dq  = cs.Function("didq", [q_sym, qdot_sym, qddot_sym], [derivative_id_expression_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})

print derivative_id_function_dq(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))


[[00, 3.22059, 0.652579, -1.94272, 0.107956, -0.174549], 
 [00, -1.80836, -3.38084, 0.531047, -0.0193066, -0.0290333], 
 [00, -0.631776, -0.326539, 0.453458, -0.0193066, -0.0290333], 
 [00, 0.452977, 0.442953, 0.405654, -0.0193066, -0.0290333], 
 [00, -0.720317, -0.720317, -0.720317, 0.118444, -0.143756], 
 [00, 0.0500702, 0.0500702, 0.0500702, -0.0813241, 0.176506]]


# cs.jtimes()

To obtain the time derivative with cs.jtimes, the same procedure as for cs.jacobian() is used with an additional variable, i.e the time derivatives of the varibales:

1. Import casadi and declare the symbolic variables needed, also the time derivatives of these: 

In [42]:
import casadi as cs

q_sym =cs.SX.sym("qs", n_joints)
qdot_sym =cs.SX.sym("qsdot", n_joints)
qddot_sym =cs.SX.sym("qsddot", n_joints)
qdddot_sym = cs.SX.sym("qsdddot", n_joints)

2. Declare the vector of the variables to find the derivatives with respect to, and the vector with their time derivatives:

In [43]:
id_vec = cs.vertcat(q_sym, qdot_sym, qddot_sym)
id_dvec = cs.vertcat(qdot_sym, qddot_sym, qdddot_sym)

3. Obtain the symbolic expression of the time derivative of ID with respect to q, qdot, and qddot using cs.jtimes():

In [44]:
timederivative_id_expression = cs.jtimes(tau_sym(q_sym, qdot_sym, qddot_sym), id_vec, id_dvec)

4. Use the symbolic expression to make a CasADi function that can be efficiently numerical evaluated:

In [54]:
timederivative_id_function = cs.Function("didtimes", [q_sym, qdot_sym, qddot_sym, qdddot_sym], [timederivative_id_expression], {"jit": True, "jit_options":{"flags":"-Ofast"}})
print timederivative_id_function(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))

[1.07229, -1.64861, 3.07646, 1.97812, -1.62826, 0.314129]


And one can also just find the time derivative of the inverse dynamics with respect to, for instance, q:

In [56]:
timederivative_id_expression_dq = cs.jtimes(tau_sym(q_sym, qdot_sym, qddot_sym), q_sym, qdot_sym)
timederivative_id_function_dq = cs.Function("dqidtimes", [q_sym, qdot_sym, qddot_sym], [timederivative_id_expression_dq], {"jit": True, "jit_options":{"flags":"-Ofast"}})
print timederivative_id_function_dq(np.ones(n_joints), np.ones(n_joints), np.ones(n_joints))

[1.86385, -4.7065, -0.553197, 1.25324, -2.18626, 0.245392]
